<a href="https://colab.research.google.com/github/sngo/llms-practice/blob/main/Company_Brochure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import userdata

In [5]:
api_key = userdata.get('OPENAI_API_KEY')

In [6]:
if api_key and api_key.startswith('sk-proj-') and len(api_key) >= 10:
  print("API key is good")
else:
  print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI(api_key=api_key)

API key is good


In [7]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
      website object
    """
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [ ]:
ed = Website("https://edwarddonner.com")
ed.links

In [9]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
link_system_prompt

In [11]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
get_links_user_prompt(ed)

In [13]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
hugginface = Website("https://huggingface.co")
hugginface.links

In [15]:
get_links(hugginface.url)

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}

In [16]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    #print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [17]:
#print(get_all_details("https://huggingface.co"))

In [18]:
system_prompt = "You are an assistant tasked with analyzing multiple relevant pages from a company’s website to create a concise and engaging brochure for prospective customers, investors, and job seekers. Your response should be in markdown format.\n"

system_prompt += "If available, include information about:\n"

system_prompt += "The company’s mission, values, and culture\n"

system_prompt += "Key customers, industries served, or notable partnerships\n"

system_prompt += "Career opportunities, work environment, or hiring approach\n"

system_prompt += "Generate two separate brochures—one in English and one in Spanish—using the following format:\n"
system_prompt += "English:\n"

system_prompt += "[Insert English brochure content here]\n"

system_prompt += "===============================\n"

system_prompt += "Spanish:\n"

system_prompt += "[Insert Spanish brochure content here]\n"

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [19]:
print(system_prompt)

You are an assistant tasked with analyzing multiple relevant pages from a company’s website to create a concise and engaging brochure for prospective customers, investors, and job seekers. Your response should be in markdown format.
If available, include information about:
The company’s mission, values, and culture
Key customers, industries served, or notable partnerships
Career opportunities, work environment, or hiring approach
Generate two separate brochures—one in English and one in Spanish—using the following format:
English:
[Insert English brochure content here]
Spanish:
[Insert Spanish brochure content here]



In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")


# Hugging Face Brochure

## English:
### Welcome to Hugging Face!
At Hugging Face, our mission is to foster collaboration in the AI and machine learning community to build innovative technologies that will shape the future. We envision a world where everyone can contribute to advancing AI in a responsible and sustainable way.

### Our Values
- **Openness**: We believe in open-source collaboration to democratize AI.
- **Community**: Our platform is built by and for the community, encouraging collective growth.
- **Innovation**: We drive creativity by enabling real-time collaboration on models, datasets, and applications.

### Who We Serve
More than 50,000 organizations leverage Hugging Face, including:
- **AI at Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

We serve diverse industries including technology, finance, healthcare, and education, providing powerful tools to accelerate their AI initiatives.

### Career Opportunities
Join a dynamic team that prioritizes a collaborative and inclusive work culture. We offer a variety of career opportunities across different departments in a fast-paced environment that values creativity and innovation.

### Work Environment
Our team thrives on innovation, collaboration, and mutual respect. We embrace diversity and encourage individuals from all backgrounds to contribute their unique perspectives.

### Hiring Approach
We seek talented individuals who are passionate about AI and machine learning. If you are ready to help us build the future, explore our open positions today!

---

## Spanish:
### ¡Bienvenido a Hugging Face!
En Hugging Face, nuestra misión es fomentar la colaboración en la comunidad de IA y aprendizaje automático para construir tecnologías innovadoras que darán forma al futuro. Imaginamos un mundo donde todos pueden contribuir a avanzar en la IA de manera responsable y sostenible.

### Nuestros Valores
- **Apertura**: Creemos en la colaboración de código abierto para democratizar la IA.
- **Comunidad**: Nuestra plataforma está construida por y para la comunidad, fomentando el crecimiento colectivo.
- **Innovación**: Impulsamos la creatividad al habilitar la colaboración en tiempo real sobre modelos, conjuntos de datos y aplicaciones.

### A Quiénes Servimos
Más de 50,000 organizaciones aprovechan Hugging Face, incluidos:
- **IA en Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

Servimos a diversas industrias, incluidas tecnología, finanzas, salud y educación, proporcionando potentes herramientas para acelerar sus iniciativas de IA.

### Oportunidades Laborales
Únete a un equipo dinámico que prioriza una cultura de trabajo colaborativa e inclusiva. Ofrecemos una variedad de oportunidades laborales en diferentes departamentos en un entorno acelerado que valora la creatividad y la innovación.

### Ambiente Laboral
Nuestro equipo prospera en innovación, colaboración y respeto mutuo. Abrazamos la diversidad y alentamos a individuos de todos los orígenes a contribuir con sus perspectivas únicas.

### Enfoque de Contratación
Buscamos individuos talentosos que sean apasionados por la IA y el aprendizaje automático. Si estás listo para ayudarnos a construir el futuro, ¡explora nuestras posiciones abiertas hoy!



In [25]:
#stream_brochure("CNN", "https://www.cnn.com/")

In [26]:
stream_brochure("Wellsfargo", "https://www.wellsfargo.com/")


# English Brochure: Wells Fargo

## Our Mission
At Wells Fargo, our mission is to satisfy our customers' financial needs and help them succeed financially. We aim to provide a range of banking, investment, mortgage, and insurance products that enable customers to achieve their financial goals.

## Our Values and Culture
Wells Fargo is built on a foundation of integrity, responsibility, and transparency. We value:
- **Diversity**: A commitment to creating an inclusive environment.
- **Customer Focus**: Placing customers at the heart of everything we do.
- **Innovation**: Encouraging creativity to solve clients' financial challenges.

## Key Customers and Industries
We proudly serve:
- Individuals and Families
- Small Businesses
- Large Corporations
- Government Entities

Wells Fargo’s extensive network of services includes personal banking, commercial banking, corporate and investment banking, insurance, and wealth management.

## Notable Partnerships
Wells Fargo has established partnerships with various organizations to enhance customer experiences, including collaboration with tech companies for improved digital banking solutions.

## Career Opportunities and Work Environment
Join our dynamic team at Wells Fargo! We offer:
- **Career Growth**: Opportunities for advancement through training and development.
- **Inclusive Workplace**: A supportive environment that encourages collaboration and creativity.
- **Comprehensive Benefits**: Competitive salaries and extensive benefits packages.

We are looking for individuals who are passionate about providing exceptional service and are eager to contribute to our mission.

## Contact Us
Explore more about our services and career opportunities at [Wells Fargo Careers](https://www.wellsfargo.com/careers).

===============================

# Spanish Brochure: Wells Fargo

## Nuestra Misión
En Wells Fargo, nuestra misión es satisfacer las necesidades financieras de nuestros clientes y ayudarles a tener éxito financieramente. Nuestro objetivo es proporcionar una variedad de productos de banca, inversión, hipotekas y seguros que permitan a los clientes alcanzar sus objetivos financieros.

## Nuestros Valores y Cultura
Wells Fargo se basa en una fundación de integridad, responsabilidad y transparencia. Valoramos:
- **Diversidad**: Un compromiso con la creación de un entorno inclusivo.
- **Enfoque en el Cliente**: Colocar a los clientes en el corazón de todo lo que hacemos.
- **Innovación**: Fomentar la creatividad para resolver los desafíos financieros de los clientes.

## Clientes Clave e Industrias
Nos enorgullece servir a:
- Individuos y Familias
- Pequeñas Empresas
- Grandes Corporaciones
- Entidades Gubernamentales

La extensa red de servicios de Wells Fargo incluye banca personal, banca comercial, banca corporativa e inversión, seguros y gestión de patrimonio.

## Asociaciones Notables
Wells Fargo ha establecido asociaciones con varias organizaciones para mejorar las experiencias de los clientes, incluyendo colaboración con empresas tecnológicas para mejorar las soluciones de banca digital.

## Oportunidades de Carrera y Ambiente Laboral
¡Únete a nuestro equipo dinámico en Wells Fargo! Ofrecemos:
- **Crecimiento Profesional**: Oportunidades de avance a través de capacitación y desarrollo.
- **Lugar de Trabajo Inclusivo**: Un entorno de apoyo que fomenta la colaboración y la creatividad.
- **Beneficios Integrales**: Salarios competitivos y paquetes de beneficios extensos.

Buscamos personas apasionadas por ofrecer un servicio excepcional y que estén listas para contribuir a nuestra misión.

## Contáctanos
Explora más sobre nuestros servicios y oportunidades profesionales en [Wells Fargo Carreras](https://www.wellsfargo.com/careers).


In [27]:
stream_brochure("Toppan Merril", "https://www.toppanmerrill.com/")


# Toppan Merrill Brochure

## Mission, Vision, and Values
At Toppan Merrill, our mission is to **simplify the complexity of regulatory disclosure and compliant communications**. We empower our clients with innovative technology and expert solutions that drive efficiency and ensure compliance in financial reporting and corporate communications. Our values center around **integrity, innovation, and client-centricity**, fostering a culture that advocates for transparency and excellence in the industry.

## Our Solutions
Toppan Merrill provides a broad range of solutions tailored to the financial services sector, including:

- **Capital Markets**: IPO, M&A, SPAC transactions
- **Corporate Compliance**: EDGAR filings, iXBRL disclosures, SOX compliance
- **Financial Services**: Regulatory disclosure, marketing communications
- **Health Plans Member Communications**: Optimized document management

### Key Technologies
Our platforms include:
- **Bridge**: Streamlined EDGAR & iXBRL filing.
- **SEC Connect**: Efficient management of various SEC forms.
- **SOX Automation**: Comprehensive program management.
- **Control Center**: Enhanced content lifecycle management.

## Key Customers and Industries Served
For over 50 years, Toppan Merrill has partnered with **industry leaders** in capital markets, finance, and healthcare. Our commitment extends to corporations navigating the intricacies of regulatory compliance and effective communication to their stakeholders.

## Career Opportunities
We believe our employees are vital to our success. Toppan Merrill seeks passionate, innovative, and talented individuals to join our team. Our hiring approach emphasizes:

- **Diversity and Inclusion**: We celebrate varied backgrounds and perspectives.
- **Professional Growth**: We provide resources for ongoing learning and development.
- **Collaborative Environment**: Our team culture promotes collaboration and support.

Explore exciting career opportunities at Toppan Merrill and be part of a dynamic team dedicated to advancing business and expanding possibilities.

## Get in Touch
Transform your regulatory compliance and communication needs with Toppan Merrill. **Contact us** today to discover how we can help streamline your processes and enhance your engagements.

---

# Toppan Merrill Folleto

## Misión, Visión y Valores
En Toppan Merrill, nuestra misión es **simplificar la complejidad de la divulgación regulatoria y las comunicaciones conformes**. Empoderamos a nuestros clientes con tecnología innovadora y soluciones expertas que impulsan la eficiencia y garantizan el cumplimiento en la presentación de informes financieros y comunicaciones corporativas. Nuestros valores se centran en **integridad, innovación y enfoque en el cliente**, fomentando una cultura que aboga por la transparencia y la excelencia en la industria.

## Nuestras Soluciones
Toppan Merrill ofrece una amplia gama de soluciones adaptadas al sector de servicios financieros, incluyendo:

- **Mercados de Capital**: Ofertas públicas iniciales, fusiones y adquisiciones, transacciones SPAC
- **Cumplimiento Corporativo**: Presentaciones EDGAR, divulgaciones iXBRL, cumplimiento de SOX
- **Servicios Financieros**: Divulgación regulatoria, comunicaciones de marketing
- **Comunicaciones para Miembros de Planes de Salud**: Gestión óptima de documentos

### Tecnologías Clave
Nuestras plataformas incluyen:
- **Bridge**: Presentación simplificada de EDGAR e iXBRL.
- **SEC Connect**: Gestión eficiente de varios formularios de la SEC.
- **Automatización SOX**: Gestión integral del programa.
- **Centro de Control**: Gestión mejorada del ciclo de vida del contenido.

## Principales Clientes e Industrias Servidas
Durante más de 50 años, Toppan Merrill ha colaborado con **líderes de la industria** en mercados de capital, finanzas y atención médica. Nuestro compromiso se extiende a las corporaciones que navegan por las complejidades del cumplimiento regulatorio y la comunicación efectiva con sus partes interesadas.

## Oportunidades de Carrera
Creemos que nuestros empleados son vitales para nuestro éxito. Toppan Merrill busca personas apasionadas, innovadoras y talentosas para unirse a nuestro equipo. Nuestro enfoque de contratación enfatiza:

- **Diversidad e Inclusión**: Celebramos diversos orígenes y perspectivas.
- **Crecimiento Profesional**: Proporcionamos recursos para el aprendizaje y desarrollo continuos.
- **Entorno Colaborativo**: Nuestra cultura de equipo promueve la colaboración y el apoyo.

Explora emocionantes oportunidades de carrera en Toppan Merrill y sé parte de un equipo dinámico dedicado a avanzar en los negocios y expandir posibilidades.

## Contáctanos
Transforma tus necesidades de cumplimiento regulatorio y comunicación con Toppan Merrill. **Contáctanos** hoy para descubrir cómo podemos ayudar a simplificar tus procesos y mejorar tus interacciones.
